In [9]:
import laspy as lp
import numpy as np
import matplotlib.pyplot as plt


def read_las_file(las_path: str):
    try:
        return lp.read(las_path)
    except Exception as e:
        print(f"Error reading {las_path}: {e}")
        return None

def normalize_points(points, eps= 1e-8):
    if not isinstance(points, np.ndarray) or points.shape[1] != 3:
        raise ValueError("points must be a numpy array with shape (n, 3).")
    points_ = points.copy()

    x_min, x_max = np.min(points_[:,0]), np.max(points_[:,0])
    y_min, y_max = np.min(points_[:,1]), np.max(points_[:,1])
    z_min, z_max = np.min(points_[:,2]), np.max(points_[:,2])

    points_[:,0] = (points_[:,0] - x_min) / (x_max - x_min + eps)
    points_[:,1] = (points_[:,1] - y_min) / (y_max - y_min + eps)
    points_[:,2] = (points_[:,2] - z_min) / (z_max - z_min + eps)
    
    return points_, x_min, x_max, y_min, y_max, z_min, z_max  
   
def normalize_colors(colors):
    rgba = colors - colors.min(axis=0)
    rgba = rgba / rgba.max(axis=0)
    return rgba #(rgba * 255).astype(np.uint8)

# pozas a analizar
# 05-04-2025, PAM4:6, RBN, RBS, 1B, 2B, 3B  (100-150m_10ms_100khz, 150m_7ms_100khz)
# 06-04-2025, 6B,7B,8B (100-150m_10ms_100khz)
# 07-04-2025 9D,9E,9F (07-04-25 100m&150m 100Khz 10ms)
# 20-05-2025 7H-7I (250520_164647_7H-7I)

rasantes = {
    'km-11': 2300.382,  
    'km-12': 2300.345,   
    'km-13': 2300.372,
    '3-a': 2300.305,
    '2-a': 2300.700,
    'pam-2': 2300.618  
}

salmueras = {
    'km-11': 2300.866,
    'km-12': 2300.582,
    'km-13': 2300.647,
    'pam-2': 2302.185,
    '2-a': 2301.129,
    '3-a': 2300.978
}

pozas = {
    "05-04-2025": [
        (558911.202,	7395022.839,	2302.254),
        (559066.708,	7395213.229,	2302.312),
        (559426.535,	7395214.473,	2302.104),
        (559424.764,	7394923.6,	    2302.141),
        (559086.394,	7394922.151,	2302.548)
    ],
    "06-04-2025": [
        (560911.613,	7394924.804,	2302.257),
        (560890.692,	7395213.724,	2302.17),
        (560226.493,	7395213.41,	    2302.125),
        (560272.775,	7394923.702,	2302.166)
    ],
    "07-04-2025": [
        (560017.361,	7396867.478,	2302.276),
        (560337.286,	7396821.869,	2302.183),
        (560336.371,	7397452.667,	2302.161),
        (560014.374,	7397467.77,	    2302.101)
    ],
    "20-05-2025": [
        (559312.555,	7397597.671,	2302.237),
        (559191.918,	7397634.896,	2300.614),
        (559181.084,	7397829.565,	2300.549),
        (559404.816,	7397855.527,	2300.577),
        (559420.061,	7397653.057,	2300.549),
        (559401.47,	    7397565.702,	2301.48),
        (559436.115,	7397321.4,	    2301.522),
        (559167.14,	    7397328.512,	2301.265),
        (559163.896,	7397558.826,	2301.331)
    ]
}


#DATE = '250329_153004'
DATE = '250329_171605'
#DATE ='250329_162159'
#DATE = '250329_105110'
#DATE = '250330_160317'
#DATE = '250330_142031'
H = 100
V = 7
F = 100
C = 'plena-luz'
T = 'clasificadas_sin_talud'
POZA = 'km-11'
S = 'sector-a'
PATH_LAS = f'/home/diego/Downloads/{DATE}_{H}m_{V}ms_{F}khz_{C}/{T}/{POZA}-{S}_{H}m_{V}ms_{F}khz_{C}_0_0.las' 
#PATH_LAS = f'/home/diego/Downloads/{DATE}_{H}m_{V}ms_{F}khz_{C}/{T}/{POZA}_{H}m_{V}ms_{F}khz_{C}_0_0.las' 
print(PATH_LAS)

LIMIT = -1

rasante = rasantes.get(POZA, None)
c_salmuera_sqm = salmueras.get(POZA, None)
i_puntos = pozas.get(POZA,None)

print(f'{POZA}\nRasante: {rasante}\nCota Salmuera SQM: {c_salmuera_sqm}\nPuntos Control: {i_puntos}')

/home/diego/Downloads/250329_171605_100m_7ms_100khz_plena-luz/clasificadas_sin_talud/km-11-sector-a_100m_7ms_100khz_plena-luz_0_0.las
km-11
Rasante: 2300.382
Cota Salmuera SQM: 2300.866
Puntos Control: [(565069.97, 7403193.413, 2300.565), (565191.968, 7403283.869, 2300.542), (565195.699, 7403102.58, 2300.527)]


In [10]:
las = read_las_file(PATH_LAS)

points = np.vstack((las.x, las.y, las.z)).transpose()
data_points = np.hstack((points,
                        np.expand_dims(las.return_number, -1),
                        np.expand_dims(las.number_of_returns, -1)))

#first = data_points[(data_points[:,-2] == 1) & (data_points[:,-1] == 2)][:, :3]
one_return = data_points[(data_points[:, -1] == 1)][:, :3]

x, y, z = one_return[:,0], one_return[:,1], one_return[:,2] 


print(f'Cota Sal promedio {POZA}: {np.mean(z[z<c_salmuera_sqm])}')


Cota Sal promedio km-11: 2300.6519113569666


In [11]:
import numpy as np
from sklearn.mixture import GaussianMixture
import pandas as pd

def find_best_gmm(data, max_components=2, criterion='aic', **kwargs):
    '''
    Find the best number of components for a Gaussian Mixture Model using AIC or BIC. 
    '''

    n_components_range = range(1, max_components + 1)
    criterions, means = [], []

    if criterion not in ['aic', 'bic']:
        raise ValueError("Invalid criterion. Use 'bic' or 'aic'.")

    for n in n_components_range:
        try:
            gmm = GaussianMixture(n_components=n, **kwargs)
            gmm.fit(data)
            if criterion == 'bic':
                criterions.append(gmm.bic(data))
            else:
                criterions.append(gmm.aic(data))
            means.append(gmm.means_)
        except ValueError as e:
            print(f"Error fitting GMM with {n} components: {e}")
            break

    return n_components_range[np.argmin(criterions)], means[np.argmin(criterions)]


In [16]:
import numpy as np
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

points = np.vstack((x, y)).transpose()
kdtree = KDTree(points)
radius = 4

for idx, (xi, yi, cota_sal_sqm) in enumerate(i_puntos, start=1):
    point_of_interest = np.array([xi, yi])
    indices = kdtree.query_ball_point(point_of_interest, radius)
    selected_z = z[indices]
    # z diffs
    diffs = np.abs(selected_z - cota_sal_sqm)
    # find the index of the minimum difference
    if len(diffs) == 0:
        cota_sal_min_error = np.nan
        distance = np.nan
    else:
        min_index = np.argmin(diffs)
        # get the corresponding z value
        cota_sal_min_error = selected_z[min_index]
        distance = np.linalg.norm(points[indices][min_index] - point_of_interest)
    # Clip by salt level
    clipped_z = selected_z[selected_z >= rasante]
    cota_sal_promedio_original = np.mean(selected_z)
    cota_sal_promedio_clipped = np.mean(clipped_z)
    #n, cota_sal = find_best_gmm(np.expand_dims(z[indices],-1), max_components=1)
    print(f'Cota sal SQM: {cota_sal_sqm}')
    print(f'Cota sal promedio original: {cota_sal_promedio_original}')
    print(f'Cota sal promedio clipped: {cota_sal_promedio_clipped}')

    # find the nearest point to the point of interest
    nearest_index = kdtree.query(point_of_interest)[1]
    nearest_point = points[nearest_index]
    nearest_z = z[nearest_index]
    min_distance = np.linalg.norm(nearest_point - point_of_interest)
    #print(f'Nearest point: {nearest_point}')
    print(f'Z del punto más cercano al punto de interés: {nearest_z}', f'Distancia: {min_distance}')
    print(f'Cota sal promedio con error mínimo: {cota_sal_min_error}', f'Distancia: {distance}')
    print("")

Cota sal SQM: 2300.565
Cota sal promedio original: 2300.6515224762197
Cota sal promedio clipped: 2300.6515224762197
Z del punto más cercano al punto de interés: 2300.661 Distancia: 0.02862254325159432
Cota sal promedio con error mínimo: 2300.56475 Distancia: 2.639957007094857

Cota sal SQM: 2300.542
Cota sal promedio original: nan
Cota sal promedio clipped: nan
Z del punto más cercano al punto de interés: 2300.7617499999997 Distancia: 76.77038086485094
Cota sal promedio con error mínimo: nan Distancia: nan

Cota sal SQM: 2300.527
Cota sal promedio original: nan
Cota sal promedio clipped: nan
Z del punto más cercano al punto de interés: 2300.61425 Distancia: 80.93915617340677
Cota sal promedio con error mínimo: nan Distancia: nan

